In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../")
import warnings
warnings.filterwarnings("ignore")

In [2]:
import json
import glob
from tqdm.notebook import trange, tqdm
import numpy as np
import pandas as pd
from features import prepare_entry, get_transformer, merge_entries
from utills import chunker, compress_fraction
import pickle
from matplotlib import rcParams
import matplotlib.pyplot as plt
import matplotlib.style as style
from pushshift_api import get_user_comments
from datetime import datetime
%matplotlib inline

import networkx as nx
from plotly.offline import init_notebook_mode
import plotly.offline as py
import plotly.graph_objs as go
init_notebook_mode(connected=True)

In [3]:
from config import known_bots

In [4]:
tqdm.pandas()

In [5]:
DATA_DIR = '../data/'
TEMP_DIR = '../temp_data/'
# DATA_PATH = DATA_DIR + 'gamestop/comments.jsonl'
DATA_PATH = DATA_DIR + 'sockpuppets/r_gaming/comments.jsonl'
TEMP_DATA_PATH = TEMP_DIR + 'sockpuppets/r_gaming/'
MULTIDOC_MODEL_PATH = TEMP_DIR + 'reddit/multidoc_20/model_20.p'
SINGLEDOC_MODEL_PATH = TEMP_DIR + 'reddit/unchunked/model.p'
chunk_sz = 20

Load Predictions From HPC
===

In [6]:
with open(TEMP_DATA_PATH + 'vectorizing_parameters.p', 'rb') as f:
    (author_order, total_chunks, author_bounds, author_to_idx_nc, x_shape, x_shape_nc) = pickle.load(f)

In [7]:
len(author_bounds)

1110

In [11]:
user_pairs = []
probs_nc = []

inter_probs_mean = []
inter_probs_std = []

intraA_probs_mean = []
intraA_probs_std = []

intraB_probs_mean = []
intraB_probs_std = []
pred_lengths = []
    
for fp in tqdm(result_files):
    with open(fp, 'rb') as f:
        try:
            (
                local_probs_nc,
                local_inter_probs_mean,
                local_inter_probs_std,
                local_intraA_probs_mean,
                local_intraA_probs_std,
                local_intraB_probs_mean,
                local_intraB_probs_std,
                local_pred_lengths,
                _, 
                local_user_pairs
            ) = pickle.load(f)
            
            probs_nc.extend(local_probs_nc)
            inter_probs_mean.extend(local_inter_probs_mean)
            inter_probs_std.extend(local_inter_probs_std)
            intraA_probs_mean.extend(local_intraA_probs_mean)
            intraA_probs_std.extend(local_intraA_probs_std)
            intraB_probs_mean.extend(local_intraB_probs_mean)
            intraB_probs_std.extend(local_intraB_probs_std)
            pred_lengths.extend(local_pred_lengths)
            user_pairs.extend(local_user_pairs)
        except:
            continue

In [12]:
user_pairs = np.array(user_pairs)

In [11]:
len(user_pairs)

615495

In [13]:
inter_probs_mean = np.array(inter_probs_mean)
intraA_probs_mean = np.array(intraA_probs_mean)
intraB_probs_mean = np.array(intraB_probs_mean)
inter_probs_std = np.array(inter_probs_std)
intraA_probs_std = np.array(intraA_probs_std)
intraB_probs_std = np.array(intraB_probs_std)
pred_lengths = np.array(pred_lengths)


probs_nc = np.array(probs_nc)

n_a = pred_lengths[:, 0]
n_b = pred_lengths[:, 1]
n_ab = pred_lengths[:, 2]

intra_probs_mean = (intraA_probs_mean * n_a + intraB_probs_mean * n_b)/ (n_a + n_b)
intra_probs_std = (
        n_a * (intraA_probs_std ** 2 + (intraA_probs_mean - intra_probs_mean)**2) + 
        n_b * (intraB_probs_std ** 2 + (intraB_probs_mean - intra_probs_mean)**2)
    ) / (n_a + n_b)


pooled_mean = (intra_probs_mean * (n_a + n_b) + inter_probs_mean * n_ab)/ (n_a + n_b + n_ab)
pooled_std = (
        (n_a + n_b) * (intra_probs_mean ** 2 + (intra_probs_mean - pooled_mean)**2) + 
        n_ab * (inter_probs_mean ** 2 + (inter_probs_mean - pooled_mean)**2)
    ) / (n_a + n_b + n_ab)


In [14]:
inter_intra_sim = (1 - np.abs(inter_probs_mean - intra_probs_mean))
aggr_score = (probs_nc * (1 - np.abs(inter_probs_mean - intra_probs_mean)))

In [25]:
selected_mask = aggr_score > 0.995

In [27]:
selected_mask.sum(), len(selected_mask)

(0, 615495)